In [1]:
import cv2
import numpy as np

In [2]:
template = cv2.imread('template_1.jpg', 0)
#cv2.imshow('template', template)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [3]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [4]:
keypoints_template, descriptors_template = sift.detectAndCompute(template, None)

In [5]:
cap = cv2.VideoCapture(0)

In [6]:
while cap.isOpened():
    
    ret, frame = cap.read()
    
    if not ret:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    keypoints_frame, descriptors_frame = sift.detectAndCompute(frame, None) 
    
    matches = bf.knnMatch(descriptors_template, descriptors_frame, k=2)
    
    good_matches = []
    
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
            
    if len(good_matches) >= 4:
        
        src_pts = np.float32([keypoints_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2) 
        dst_pts = np.float32([keypoints_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        M,_=cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
        
        h, w = template.shape
        
        template_corner = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        transformed_corner = cv2.perspectiveTransform(template_corner, M)
        
        bounding_box = cv2.polylines(frame, [np.int32(transformed_corner)], True, (0, 255, 0), 2)
        cv2.imshow('bounding box', bounding_box)
            
    
    if cv2.waitKey(1)==27:
        break
cap.release()
cv2.destroyAllWindows()